In [6]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
%matplotlib inline

np.set_printoptions(formatter={'all': lambda x: '%.4f' % x})

In [7]:
def hessian(fn: Callable, point: np.ndarray, delta: float): 
    var_count = len(point)
    fn_2dx = lambda fn, p, delta: (fn(p[0] + delta, p[1]) - 2 * fn(*p) + fn(p[0] - delta, p[1])) / (delta ** 2)
    fn_2dy = lambda fn, p, delta: (fn(p[0], p[1] + delta) - 2 * fn(*p) + fn(p[0], p[1] - delta)) / (delta ** 2)
    fn_dxy = lambda fn, p, delta: (fn(p[0] + delta, p[1] + delta) - fn(p[0] + delta, p[1] - delta) - fn(p[0] - delta, p[1] + delta) + fn(p[0] - delta, p[1] - delta)) / 4 * (delta ** 2)
    h = np.zeros((var_count, var_count))
    h[0, 0] = fn_2dx(fn, point, delta)
    h[1, 1] = fn_2dy(fn, point, delta)
    h[0, 1] = fn_dxy(fn, point, delta)
    h[1, 0] = h[0, 1]
    return h

def gradient(fn: Callable, p: np.ndarray, delta):
    fn_dx = lambda fn, p, delta: (fn(p[0] + delta, p[1]) - fn(p[0] - delta, p[1])) / (2 * delta)
    fn_dy = lambda fn, p, delta: (fn(p[0], p[1] + delta) - fn(p[0], p[1] - delta)) / (2 * delta)
    return np.array([fn_dx(fn, p, delta), fn_dy(fn, p, delta)])

def levenberg_marquardt(
    fn: Callable,
    point: np.ndarray,
    e1: float,
    e2: float,
    delta: float,
    alpha: float = 0.05,
    max_iters: int = 1000,
    debug: bool = False
):
    x = point

    for iter in range(max_iters):
        fn_prev = fn(*x)

        if debug and iter == 0:
            print(f'f({x[0], x[1]}) = {fn_prev}')
            print('Iter\t fn(x, y)\t||grad||')

        grad = gradient(fn, x, delta)
        h = hessian(fn, x, delta)
        s = np.matmul(-1 * np.linalg.inv(h + alpha * np.eye(len(x))), np.transpose(grad))

        x = x + np.transpose(s) 
        fn_curr = fn(*x)

        alpha = alpha / 2 if fn_curr < fn_prev else 2 * alpha

        if debug:
            print(f'{iter + 1}\tf({x[0]:.2f},{x[1]:.2f}) = {fn_curr:.2f}\t{np.linalg.norm(grad):.2f}')

        if abs(fn_curr - fn_prev) <= e1 and np.linalg.norm(grad) <= e2:
            return { 'point': x, 'iters': iter + 1 }
    
    return { 'point': x, 'iters': max_iters }

In [11]:
fn2 = lambda x, y : (x - 3) ** 2 + (y - 5) ** 2
x = np.array([7, 9])

levenberg_marquardt(
    fn2,
    x,
    e1 = 0.001,
    e2 = 0.002,
    alpha = 0.001,
    delta = 1
)

{'point': array([3.0000, 5.0000]), 'iters': 3}

In [10]:
fn2 = lambda x, y :190 * (np.sqrt(x ** 2 + (y + 1) ** 2) - 1) ** 2 - (20 * x + 40 * y)
x = np.array([7, 9])

levenberg_marquardt(
    fn2,
    x,
    e1 = 0.001,
    e2 = 0.002,
    alpha = 0.001,
    delta = 1
)

{'point': array([0.1696, 0.0427]), 'iters': 18}